In [1]:
import pandas as pd
import numpy as np
import altair as alt
#import matplotlib as plt
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import norm

import chart_studio
import chart_studio.plotly as py

In [2]:
seasons = range(2020, 2014, -1)
pos_files = ['Defenses', 'Kickers', 'Quarterbacks', 'RunningBacks', 'TightEnds', 'WideReceivers']
pos_dict = {'Defenses': 'DSt', 'Kickers': 'K', 'Quarterbacks': 'QB', 'RunningBacks': 'RB', 'TightEnds': 'TE', 'WideReceivers': 'WR'}

In [3]:
season = seasons[0]
df_fwar = pd.read_csv('data/'+str(season)+'/FantasyWinsAboveReplacement_PPR.csv', sep=';', decimal=',', index_col=0).sort_values('FWAR', ascending=False)
All_Players_2020 = pd.read_csv('data/'+str(season)+'/All_Players_Fts_ppr.csv', sep=';', decimal=',', index_col=0)

In [4]:
df_fwar.head()

,player,team,position,FP_total,FP_meanGame,FPAX_total,FPAX_meanGame,FWins_IfFull_added,FWins_actual_added,FWAR
27,Alvin Kamara,NO,RB,375.8,25.053,142.496,9.500,9.478,9.478,3.669
663,Travis Kelce,KC,TE,310.6,20.707,99.260,6.617,9.160,9.160,2.861
165,Dalvin Cook,MIN,RB,327.8,23.414,108.868,7.776,9.230,8.615,2.806
215,Derrick Henry,TEN,RB,294.1,19.607,63.729,4.249,8.311,8.311,2.502
622,Stefon Diggs,BUF,WR,314.0,20.933,29.766,1.984,8.099,8.099,2.095


In [5]:
fwpctAx_2020 = All_Players_2020.sort_values('FWPctAX', ascending=False)
fwpctAx_2020.head(10)

,player,team,position,week,FP,FPAX,FWPctAX
4530,Alvin Kamara,NO,RB,16,56.2,39.086,0.920
532,Tyreek Hill,KC,WR,12,57.9,38.781,0.885
5300,Alvin Kamara,NO,RB,10,34.8,19.828,0.885
5116,Dalvin Cook,MIN,RB,8,48.6,34.164,0.884
48,Davante Adams,GB,WR,1,41.6,22.392,0.877
5191,Dalvin Cook,MIN,RB,9,39.2,25.872,0.868
2757,Darren Waller,LV,TE,13,45.0,27.900,0.860
1620,Chase Claypool,PIT,WR,5,42.6,23.089,0.851
5190,Christian McCaffrey,CAR,RB,9,37.1,23.772,0.848
3752,Josh Jacobs,LV,RB,1,35.9,19.656,0.846


### Ideas

- Correlation FWAR between innerhalb jedes Teams: QB1-WR1, WR2, WR3, RB1, RB2, TE1 (over Season)
- Sum up FPAX / Week
- t-Test (evtl nur Top-Spieler? aber, z.B. Mahomes+Hill vs Mahomes+Diggs)
- Prediction von Stack von Season auf Season+1
- Overall: welche Stacks-Kombis sinnvoll?
- Overall: welche Team sinnvolle Stacks?

##### Goal
- welche stack sind sinnvoll, welche ? QB+WR/RB/TE oder vlt 2 aus WR/RB/TE? (bekomme aus Korrelation)
compare, if stacking a Duo(/Trio) of a team is more useful than not.
- over whole season -> FWAR
- weekly -> WPct
Problem: was heißt NOT? Base-Team? Avg von Top-12/24? Mittel aus allen anderen Pct von Spielern kombiniert mit QB


##### Advanced
- Draft Strategy: Reach for Stacks?





In [96]:
# within Teams

cols = ['Team', 'Season', 'QB', 'RB1', 'RB2', 'WR1', 'WR2', 'WR3', 'TE1', 'Dst']
df_corr = pd.DataFrame(columns=cols)
teams = set(df_fwar['team'])

for season in seasons:
    df_fwar = pd.read_csv('data/'+str(season)+'/FantasyWinsAboveReplacement_PPR.csv', sep=';', decimal=',', index_col=0).sort_values('FWAR', ascending=False)
    df_fwar.drop(columns=['FP_total','FP_meanGame', 'FPAX_total', 'FWins_IfFull_added'], inplace=True)
    for team in set(df_fwar['team']):
        df_team = df_fwar[df_fwar['team'] == team]
        te1 = df_team[df_team['position']=='TE'].FWAR.iloc[0]
        qb  = df_team[df_team['position']=='QB'].FWAR.iloc[0]
        rb1 = df_team[df_team['position']=='RB'].FWAR.iloc[0]
        rb2 = df_team[df_team['position']=='RB'].FWAR.iloc[1]
        wr1 = df_team[df_team['position']=='WR'].FWAR.iloc[0]
        wr2 = df_team[df_team['position']=='WR'].FWAR.iloc[1]
        wr3 = df_team[df_team['position']=='WR'].FWAR.iloc[2]
        dst = df_team[df_team['position']=='DSt'].FWAR.iloc[0]
    
        df_corr = df_corr.append(pd.DataFrame([[team, season, qb, rb1, rb2, wr1, wr2, wr3, te1, dst]], columns=cols), ignore_index=True)


# next: Correlation only for Top16 QBs
# best Teams ? oder nach was sortieren?
#df_corr_top16 = df_corr.sort_values('WR1', ascending=False)[0:32]
#df_corr_top16.corr()
df_corr.corr()

,QB,RB1,RB2,WR1,WR2,WR3,TE1,Dst
QB,1.000000,0.105669,0.101415,0.298132,0.254156,0.183498,0.163119,0.152227
RB1,0.105669,1.000000,0.168259,-0.000588,0.019065,-0.067569,-0.006913,-0.020877
RB2,0.101415,0.168259,1.000000,-0.025369,-0.119969,-0.143185,-0.000941,0.091350
WR1,0.298132,-0.000588,-0.025369,1.000000,0.493683,0.250286,-0.003338,0.112429
WR2,0.254156,0.019065,-0.119969,0.493683,1.000000,0.511607,-0.167486,0.053703
WR3,0.183498,-0.067569,-0.143185,0.250286,0.511607,1.000000,-0.024671,0.095751
TE1,0.163119,-0.006913,-0.000941,-0.003338,-0.167486,-0.024671,1.000000,0.006776
Dst,0.152227,-0.020877,0.091350,0.112429,0.053703,0.095751,0.006776,1.000000


In [97]:
df_corr

,Team,Season,QB,RB1,RB2,WR1,WR2,WR3,TE1,Dst
0,ATL,2020,0.000,0.126,-0.727,1.247,-0.432,-1.947,-0.076,-0.361
1,DET,2020,-0.331,0.035,-0.259,0.248,-1.662,-2.387,0.499,-1.111
2,NO,2020,-1.740,3.669,-0.017,-1.242,-1.727,-3.363,-0.669,0.503
3,BLT,2020,0.436,-0.146,-0.157,-0.446,-1.794,-1.974,0.137,0.617
4,WAS,2020,-4.159,0.955,0.552,0.000,-2.013,-2.747,0.500,0.487
...,...,...,...,...,...,...,...,...,...,...
187,NYG,2015,0.173,0.159,0.059,1.532,-0.392,-1.133,-1.352,-0.153
188,TB,2015,-0.073,1.279,0.391,-0.178,-2.237,-2.583,-1.231,-0.197
189,DAL,2015,-4.430,0.563,-1.976,-0.711,-1.086,-2.546,0.333,-0.621
190,CLV,2015,-3.203,0.275,0.080,0.083,-1.880,-2.555,1.466,-0.459


In [28]:
fwpctAx_2020.head()

,player,team,position,week,FP,FPAX,FWPctAX
4530,Alvin Kamara,NO,RB,16,56.2,39.086,0.920
532,Tyreek Hill,KC,WR,12,57.9,38.781,0.885
5300,Alvin Kamara,NO,RB,10,34.8,19.828,0.885
5116,Dalvin Cook,MIN,RB,8,48.6,34.164,0.884
48,Davante Adams,GB,WR,1,41.6,22.392,0.877


In [43]:
cols = ['Team', 'Week', 'QB', 'RB1', 'RB2', 'WR1', 'WR2', 'WR3', 'TE1', 'Dst']
df_corr = pd.DataFrame(columns=cols)

for team in teams:
    fwpctAx_2020_team = fwpctAx_2020[fwpctAx_2020['team']== team]
    for week in range(1,17):
        if week in set(fwpctAx_2020_team.week):
            fwpctAx_2020_team_week = fwpctAx_2020_team[fwpctAx_2020_team['week'] == week]
            te1 = fwpctAx_2020_team_week[fwpctAx_2020_team_week['position']=='TE'].FWPctAX.iloc[0]
            qb  = fwpctAx_2020_team_week[fwpctAx_2020_team_week['position']=='QB'].FWPctAX.iloc[0]
            rb1 = fwpctAx_2020_team_week[fwpctAx_2020_team_week['position']=='RB'].FWPctAX.iloc[0]
            rb2 = fwpctAx_2020_team_week[fwpctAx_2020_team_week['position']=='RB'].FWPctAX.iloc[1]
            wr1 = fwpctAx_2020_team_week[fwpctAx_2020_team_week['position']=='WR'].FWPctAX.iloc[0]
            wr2 = fwpctAx_2020_team_week[fwpctAx_2020_team_week['position']=='WR'].FWPctAX.iloc[1]
            wr3 = fwpctAx_2020_team_week[fwpctAx_2020_team_week['position']=='WR'].FWPctAX.iloc[2]
            dst = fwpctAx_2020_team_week[fwpctAx_2020_team_week['position']=='DSt'].FWPctAX.iloc[0]
    
            df_corr = df_corr.append(pd.DataFrame([[team, week, qb, rb1, rb2, wr1, wr2, wr3, te1, dst]], columns=cols), ignore_index=True)


# next: Correlation only for Top16 QBs
# best Teams ? oder nach was sortieren?
df_corr_topX = df_corr.sort_values('QB', ascending=False)
df_corr_topX.corr()

,QB,RB1,RB2,WR1,WR2,WR3,TE1,Dst
QB,1.000000,0.069407,0.073045,0.520413,0.483029,0.350552,0.287247,-0.033674
RB1,0.069407,1.000000,0.216643,-0.028922,-0.066640,-0.142583,-0.013453,0.087500
RB2,0.073045,0.216643,1.000000,-0.055475,0.032526,0.125038,0.039229,0.094890
WR1,0.520413,-0.028922,-0.055475,1.000000,0.473571,0.216009,-0.043666,-0.112027
WR2,0.483029,-0.066640,0.032526,0.473571,1.000000,0.653388,-0.053368,0.019455
WR3,0.350552,-0.142583,0.125038,0.216009,0.653388,1.000000,0.016918,-0.015239
TE1,0.287247,-0.013453,0.039229,-0.043666,-0.053368,0.016918,1.000000,-0.035854
Dst,-0.033674,0.087500,0.094890,-0.112027,0.019455,-0.015239,-0.035854,1.000000


In [44]:
df_corr_topX

,Team,Week,QB,RB1,RB2,WR1,WR2,WR3,TE1,Dst
429,TB,10,0.783,0.724,0.273,0.555,0.447,0.409,0.587,0.424
358,TEN,15,0.772,0.635,0.419,0.572,0.443,0.151,0.392,0.476
88,BUF,15,0.765,0.504,0.299,0.675,0.531,0.308,0.423,0.612
474,HST,11,0.763,0.347,0.281,0.391,0.359,0.287,0.489,0.326
187,KC,8,0.755,0.385,0.368,0.598,0.554,0.466,0.679,0.479
...,...,...,...,...,...,...,...,...,...,...
9,ATL,11,0.145,0.294,0.254,0.391,0.365,0.230,0.216,0.412
105,MIA,1,0.141,0.385,0.310,0.293,0.249,0.239,0.336,0.390
178,PIT,15,0.131,0.557,0.199,0.547,0.280,0.187,0.194,0.385
264,CHI,10,0.130,0.313,0.231,0.334,0.223,0.181,0.320,0.592
